# TUTORIAL DE DUCKDB NO CONTEXTO DO SPREADMART ORÇAMENTÁRIO

## TO DO
1. Clausulas SQL para somente criar ou deletar tabelas caso não já exisitam
1. Tratamento de erros e exceções para quando não for possível criar ou conectar na base de dados, criar ou deletar tabelas e etc.
1. 

## Imports

In [1]:
import pandas as pd
import glob
import time
import duckdb
import os

#conn = duckdb.connect() # cria base de dados in-memory

## Constants

In [2]:
DB_NAME = 'database/dadosmg.db'
CSV_PATH = 'datasets\\'

## Criar e conectar a uma base de dados persistente (em disco)

In [3]:
con = duckdb.connect(DB_NAME)

## Realizar selects diretamente no arquivos de dados e mostrar tempo de execução

In [4]:
cur_time = time.time()

# troca \\ por / para evitar erros nas strings de caminhos dos arquivos
file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}*.csv'))] 

for file in file_paths: 
    df = con.execute(f"""SELECT * FROM '{file}' """).df() # converte e armazena em um pandas dataframe

print(f"Tempo de select nos arquivos CSV: {(time.time() - cur_time):1.2f} segundos")

Tempo de select nos arquivos CSV: 10.93 segundos


## Apagar todas as tabelas caso existam

In [5]:
if con.execute("""SHOW TABLES""").fetchall():
    for table_name in con.execute("SHOW TABLES").fetchall():
        con.execute(f"""DROP TABLE {table_name[0]} """)
        print(f"Tabela {table_name} apagada.")
else:
    print(f"Não há tabelas na database {DB_NAME}")



Não há tabelas na database database/dadosmg.db


## Carregar arquivos CSV para tabelas com mesmo nome

In [6]:
file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}*.csv'))]

for file in file_paths:
    _ , tail = os.path.split(file)
    name , file_extension = os.path.splitext(tail)
    con.execute(f"""CREATE TABLE '{name}' AS SELECT * FROM read_csv_auto('{file}')""")
    print(f"Arquivo {file} carregado para tabela {name}")
     


Arquivo datasets/dm_acao.csv carregado para tabela dm_acao
Arquivo datasets/dm_categ_econ.csv carregado para tabela dm_categ_econ
Arquivo datasets/dm_elemento_desp.csv carregado para tabela dm_elemento_desp
Arquivo datasets/dm_empenho_desp_2002.csv carregado para tabela dm_empenho_desp_2002
Arquivo datasets/dm_empenho_desp_2003.csv carregado para tabela dm_empenho_desp_2003
Arquivo datasets/dm_empenho_desp_2004.csv carregado para tabela dm_empenho_desp_2004
Arquivo datasets/dm_empenho_desp_2005.csv carregado para tabela dm_empenho_desp_2005
Arquivo datasets/dm_empenho_desp_2006.csv carregado para tabela dm_empenho_desp_2006
Arquivo datasets/dm_empenho_desp_2007.csv carregado para tabela dm_empenho_desp_2007
Arquivo datasets/dm_empenho_desp_2008.csv carregado para tabela dm_empenho_desp_2008
Arquivo datasets/dm_empenho_desp_2009.csv carregado para tabela dm_empenho_desp_2009
Arquivo datasets/dm_empenho_desp_2010.csv carregado para tabela dm_empenho_desp_2010
Arquivo datasets/dm_empenho_

## Mostrar todas a tabelas de uma Database

In [7]:
if con.sql("""SHOW TABLES""").fetchall() != []:
    con.execute(f"""SHOW TABLES""")
else:
    print(f"Não há tabelas na database {DB_NAME}")

con.sql("""SHOW TABLES""").show()

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ dm_acao              │
│ dm_categ_econ        │
│ dm_elemento_desp     │
│ dm_empenho_desp_2002 │
│ dm_empenho_desp_2003 │
│ dm_empenho_desp_2004 │
│ dm_empenho_desp_2005 │
│ dm_empenho_desp_2006 │
│ dm_empenho_desp_2007 │
│ dm_empenho_desp_2008 │
│          ·           │
│          ·           │
│          ·           │
│ dm_empenho_desp_2014 │
│ dm_empenho_desp_2015 │
│ dm_empenho_desp_2016 │
│ dm_empenho_desp_2017 │
│ dm_empenho_desp_2018 │
│ dm_empenho_desp_2019 │
│ dm_empenho_desp_2020 │
│ dm_empenho_desp_2021 │
│ dm_empenho_desp_2022 │
│ dm_empenho_desp_2023 │
├──────────────────────┤
│  25 rows (20 shown)  │
└──────────────────────┘



## Descrever tabelas

In [8]:
con.sql("""DESCRIBE""").show()

┌──────────────────────┬──────────────────────┬────────────────────────────────────────────────────────────┬───────────┐
│      table_name      │     column_names     │                        column_types                        │ temporary │
│       varchar        │      varchar[]       │                         varchar[]                          │  boolean  │
├──────────────────────┼──────────────────────┼────────────────────────────────────────────────────────────┼───────────┤
│ dm_acao              │ [id_acao, ano_exer…  │ [BIGINT, BIGINT, BIGINT, VARCHAR]                          │ false     │
│ dm_categ_econ        │ [id_categ_econ, cd…  │ [BIGINT, BIGINT, VARCHAR]                                  │ false     │
│ dm_elemento_desp     │ [id_elemento, cd_e…  │ [BIGINT, BIGINT, VARCHAR]                                  │ false     │
│ dm_empenho_desp_2002 │ [id_empenho, ano_e…  │ [BIGINT, BIGINT, BIGINT, DATE, VARCHAR, VARCHAR, DOUBLE,…  │ false     │
│ dm_empenho_desp_2003 │ [id_emp

## Mostra conteúdo de uma tabela

In [9]:
TABLE_NAME = "dm_empenho_desp_2002"

con.table(TABLE_NAME).show()

┌────────────┬───────────────┬────────────┬───┬───────────────┬────────────┬───────────────────┬────────────────┐
│ id_empenho │ ano_exercicio │ nr_empenho │ … │ tipo_empenho  │ vr_empenho │ cd_uni_prog_gasto │ uni_prog_gasto │
│   int64    │     int64     │   int64    │   │    varchar    │   double   │       int64       │    varchar     │
├────────────┼───────────────┼────────────┼───┼───────────────┼────────────┼───────────────────┼────────────────┤
│     788955 │          2002 │          0 │ … │ SEM DESCRICAO │   38042.78 │                 0 │ NULL           │
│     788956 │          2002 │          0 │ … │ SEM DESCRICAO │     159.75 │                 0 │ NULL           │
│     788957 │          2002 │          0 │ … │ SEM DESCRICAO │    6873.15 │                 0 │ NULL           │
│     788958 │          2002 │          0 │ … │ SEM DESCRICAO │     1245.0 │                 0 │ NULL           │
│     788959 │          2002 │          0 │ … │ SEM DESCRICAO │  169691.73 │            

## Descrever uma tabela específica

In [10]:
TABLE_NAME = "dm_empenho_desp_2002"

con.table(TABLE_NAME).describe()

┌─────────┬───────────────────┬───────────────┬───┬────────────────────┬───────────────────┬────────────────┐
│  aggr   │    id_empenho     │ ano_exercicio │ … │     vr_empenho     │ cd_uni_prog_gasto │ uni_prog_gasto │
│ varchar │      double       │    double     │   │       double       │      double       │    varchar     │
├─────────┼───────────────────┼───────────────┼───┼────────────────────┼───────────────────┼────────────────┤
│ count   │          606150.0 │      606150.0 │ … │           606150.0 │          606150.0 │ 0              │
│ mean    │ 5399765.179965355 │        2002.0 │ … │ 10344.671022931549 │ 266.1621562319558 │ NULL           │
│ stddev  │ 2991273.945032219 │           0.0 │ … │  526242.7659111151 │ 641.9119659752052 │ NULL           │
│ min     │          788926.0 │        2002.0 │ … │                0.0 │               0.0 │ NULL           │
│ max     │        10003541.0 │        2002.0 │ … │        200000000.0 │            2544.0 │ NULL           │
│ median  

## Agregar linhas por valores de uma coluna
Aplica um agregado (opcionalmente agrupado) sobre a relação. O sistema agrupará automaticamente por qualquer coluna que não seja agregada.

In [11]:
TABLE_NAME = 'dm_empenho_desp_2002'
rel = con.sql(f"""SELECT * FROM {TABLE_NAME}""")


rel.aggregate("tipo_empenho AS 'Tipo do empenho', sum(vr_empenho), count(id_empenho)")

┌─────────────────┬────────────────────┬───────────────────┐
│ Tipo do empenho │  sum(vr_empenho)   │ count(id_empenho) │
│     varchar     │       double       │       int64       │
├─────────────────┼────────────────────┼───────────────────┤
│ ORDINÁRIO       │ 3010612218.4999986 │            267704 │
│ ESTIMADO        │ 2630540401.6100035 │            332933 │
│ GLOBAL          │ 255000184.17999998 │              5249 │
│ SEM DESCRICAO   │ 374269536.26000005 │               264 │
└─────────────────┴────────────────────┴───────────────────┘

## Concatenar dois arquivos CSV em uma só planilha

In [12]:
tbl1_name = 'dm_empenho_desp_2002'
tbl2_name = 'dm_empenho_desp_2003'
tbl_agg_name = 'dm_empenho_desp_agg'


df1 = con.sql(f"""SELECT * FROM {tbl1_name}""").df()
print('num linhas df1:', len(df1))
df2 = con.sql(f"""SELECT * FROM {tbl2_name}""").df()
print('num linhas df2:', len(df2))
print('total linhas:', len(df1) + len(df2))

df_agg = pd.concat([df1, df2])

con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """)

print('Num linhas da tabela agregada:')
print(con.sql(f"""SELECT COUNT(*) FROM {tbl_agg_name}"""))

num linhas df1: 606150
num linhas df2: 558038
total linhas: 1164188
Num linhas da tabela agregada:
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1164188 │
└──────────────┘



In [13]:
tbl_agg_name = 'dm_empenho_desp'
df_agg = pd.DataFrame()
num_linhas = 0

file_paths = [i.replace('\\', '/') for i in list(glob.iglob(f'{CSV_PATH}dm_empenho_desp_*.csv'))]

for file in file_paths:
    print(f'Lendo:', file)
    df1 = con.execute(f"""SELECT * FROM '{file}' """).df()
    df_agg = pd.concat([df_agg, df1])
    num_linhas += len(df1)
      
con.execute(f"""CREATE TABLE '{tbl_agg_name}' AS SELECT * FROM df_agg """) 

print('Total de linhas tabelas lidas:', num_linhas)
print('Total de linhas do dataframe concatenado:', len(df_agg))
print('Num linhas da tabela agregada:')
print(con.sql(f"""SELECT COUNT(*) FROM {tbl_agg_name}"""))

con.table(tbl_agg_name).show()

Lendo: datasets/dm_empenho_desp_2002.csv
Lendo: datasets/dm_empenho_desp_2003.csv
Lendo: datasets/dm_empenho_desp_2004.csv
Lendo: datasets/dm_empenho_desp_2005.csv
Lendo: datasets/dm_empenho_desp_2006.csv
Lendo: datasets/dm_empenho_desp_2007.csv
Lendo: datasets/dm_empenho_desp_2008.csv
Lendo: datasets/dm_empenho_desp_2009.csv
Lendo: datasets/dm_empenho_desp_2010.csv
Lendo: datasets/dm_empenho_desp_2011.csv
Lendo: datasets/dm_empenho_desp_2012.csv
Lendo: datasets/dm_empenho_desp_2013.csv
Lendo: datasets/dm_empenho_desp_2014.csv
Lendo: datasets/dm_empenho_desp_2015.csv
Lendo: datasets/dm_empenho_desp_2016.csv
Lendo: datasets/dm_empenho_desp_2017.csv
Lendo: datasets/dm_empenho_desp_2018.csv
Lendo: datasets/dm_empenho_desp_2019.csv
Lendo: datasets/dm_empenho_desp_2020.csv
Lendo: datasets/dm_empenho_desp_2021.csv
Lendo: datasets/dm_empenho_desp_2022.csv
Lendo: datasets/dm_empenho_desp_2023.csv
Total de linhas tabelas lidas: 12960234
Total de linhas do dataframe concatenado: 12960234
Num lin

## Fecha conexção com base de dados

In [ ]:
con.close()